<a href="https://colab.research.google.com/github/pbelevich/virtual-try-on/blob/main/virtual_try_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Virtual Try-On
This is a non-commercial educational project that uses [FrankMocap](https://github.com/facebookresearch/frankmocap) for pose estimation, [MultiGarmentNetwork](https://github.com/bharat-b7/MultiGarmentNetwork) for garment re-targeting. FrankMocap's [PyTorch3D](https://pytorch3d.org/) renderer was modified to support textures from MultiGarmentNetwork's garments.

[Sample video](https://www.shutterstock.com/video/clip-15249886-talanted-young-dancer-man-dancing-breakdance-on) was taken from [shutterstock.com](https://www.shutterstock.com/)

In [24]:
rm -rf {,.[!.],..?}*

In [25]:
!git clone --quiet -b master https://github.com/igorlaryush/frankmocap.git frankmocap

In [26]:
!git clone --quiet -b master https://github.com/igorlaryush/MultiGarmentNetwork.git MultiGarmentNetwork

In [27]:
!mv frankmocap/* .

In [28]:
!mv MultiGarmentNetwork/* .

In [29]:
!rm -rf frankmocap MultiGarmentNetwork

Install dependencies

In [30]:
!apt-get install -qq libboost-dev xvfb libosmesa6-dev

In [31]:
!pip install -q -r docs/requirements.txt

     |████████████████████████████████| 542kB 12.4MB/s 
     |████████████████████████████████| 1.9MB 29.6MB/s 
     |████████████████████████████████| 133kB 37.4MB/s 


In [32]:
!pip install -q torchgeometry chumpy opendr

In [35]:
!test -z "$(python -c "import pytorch3d" 2>&1)" || pip install -q 'git+https://github.com/igorlaryush/pytorch3d.git@stable'

In [36]:
!sh scripts/install_pose2d.sh
!sh scripts/download_data_body_module.sh

Cloning into 'lightweight-human-pose-estimation.pytorch'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 112, done.
remote: Total 112 (delta 0), reused 0 (delta 0), pack-reused 112
Receiving objects: 100% (112/112), 221.99 KiB | 9.25 MiB/s, done.
Resolving deltas: 100% (48/48), done.
--2021-07-14 15:13:32--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 104.64.31.122, 2a02:26f0:e600:582::4b21, 2a02:26f0:e600:58d::4b21
Connecting to download.01.org (download.01.org)|104.64.31.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87959810 (84M) [application/octet-stream]
Saving to: ‘checkpoint_iter_370000.pth’

checkpoint_iter_370 100%[=================

In [37]:
!mkdir -p extra_data/smpl
!test -f smplify_public_v2.zip || gdown --id 1BwNtKvArmUdy4Cpkx1seIlsHinopdaws
!unzip -n -p mpips-smplify_public_v2.zip smplify_public/code/models/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl > extra_data/smpl/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl

Downloading...
From: https://drive.google.com/uc?id=1BwNtKvArmUdy4Cpkx1seIlsHinopdaws
To: /content/mpips-smplify_public_v2.zip
53.0MB [00:00, 112MB/s]


In [38]:
!test -f Multi-Garmentdataset.zip || wget -q -nc https://datasets.d2.mpi-inf.mpg.de/MultiGarmentNetwork/Multi-Garmentdataset.zip

In [39]:
!test -d Multi-Garmentdataset || unzip -qn Multi-Garmentdataset.zip

In [40]:
!mkdir -p assets
!test -f mpips-smplify_public_v2.zip || gdown --id 1BwNtKvArmUdy4Cpkx1seIlsHinopdaws
!test -f assets/neutral_smpl.pkl || unzip -n -p mpips-smplify_public_v2.zip smplify_public/code/models/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl > assets/neutral_smpl.pkl

Install [mesh](https://github.com/MPI-IS/mesh)

In [41]:
!test -d mesh_dir || git clone --quiet https://github.com/MPI-IS/mesh.git mesh_dir

In [42]:
!cd mesh_dir && make all

----- [ mesh_package ] Installing with the interpreter /usr/local/bin/python (version 3.7.10)
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 1)) (57.1.0)
Requirement already up-to-date: numpy in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 2)) (1.21.0)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 3)) (3.4.2)
Requirement already up-to-date: scipy in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 4)) (1.7.0)
Requirement already up-to-date: pyopengl in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 5)) (3.1.5)
Requirement already up-to-date: pillow in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 6)) (8.3.1)
Requirement already up-to-date: pyzmq in /usr/local/lib/python3.7/dist-packages (from -r requirements.txt (line 7)) (22.1.0)
Requirement already u

Download sample source video

In [43]:
!mkdir -p ./sample_data
!test -f 15249886-preview.mp4 || gdown --id 1w4iQmuVP07FpHxOePkuhY2Sv3_RE8BPs -O ./sample_data/15249886-preview.mp4

Downloading...
From: https://drive.google.com/uc?id=1w4iQmuVP07FpHxOePkuhY2Sv3_RE8BPs
To: /content/sample_data/15249886-preview.mp4
100% 1.40M/1.40M [00:00<00:00, 44.7MB/s]


Run the demo

In [ ]:
!xvfb-run -a python -m demo.demo_bodymocap --input_path ./sample_data/15249886-preview.mp4 --out_dir ./mocap_output --renderer_type pytorch3d

Loading Body Pose Estimator
use_smplx False
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100% 97.8M/97.8M [00:01<00:00, 65.3MB/s]
--------------------------------------
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Visualization saved: ./mocap_output/rendered/00000.jpg
Time: 18.82 sec/frame, FPS 0.05
Processed : ./mocap_output/frames/00000.jpg
--------------------------------------
Visualization saved: ./mocap_output/rendered/00001.jpg
Time: 16.06 sec/frame, FPS 0.06
Processed : ./mocap_output/frames/00001.jpg
--------------------------------------

Show the result

In [ ]:
# source: https://stackoverflow.com/a/65273831

from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "./mocap_output/15249886-preview.mp4"

# Compressed video path
compressed_path = "./mocap_output/15249886-preview_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
! ls